# Broadcasting

Tensor supports numpy-like [broadcasting](https://numpy.org/doc/stable/user/basics.broadcasting.html#broadcasting) rules.

## For Users
An example of broadcasting when a 1-d array is added to a 2-d array:

In [1]:
from tensor_module import Tensor, Device
a = Tensor.from_list([[ 0.0,  0.0,  0.0],
                    [10.0, 10.0, 10.0],
                    [20.0, 20.0, 20.0],
                    [30.0, 30.0, 30.0]])

b = Tensor.from_list([1.0, 2.0, 3.0])
res = a + b
print(res.to_list())

res = b + a
print(res.to_list())


c = Tensor.from_list([1.0, 2.0, 3.0, 4.0])
try:
    _ = a + c # cannot be broadcasted together
except Exception as e:
    print(e)

[[1.0, 2.0, 3.0], [11.0, 12.0, 13.0], [21.0, 22.0, 23.0], [31.0, 32.0, 33.0]]
[[1.0, 2.0, 3.0], [11.0, 12.0, 13.0], [21.0, 22.0, 23.0], [31.0, 32.0, 33.0]]
Shapes of tensors must match for addition


# For Developers

## Debugging

The method `elem_at()` is used to get the value of a tensor at a specific index.

In the following example, we create a tensor `t` with shape `(3, 1)` and can 'view' it as a `(3,2)` tensor.
You can think of this as padding another column to `t`.

$\begin{bmatrix} x  \\ y  \\ z  \end{bmatrix}$ -> $\begin{bmatrix} x & x \\ y & y \\ z & z \end{bmatrix}$

In [2]:
from tensor_module import Tensor

t = Tensor.from_list([[1], [2], [3]])
shape = [3, 2]

for i in range(shape[0]):
    for j in range(shape[1]):
        idx = [i, j]
        print(idx, '=>',t.elem_at(shape, idx))

del t

[0, 0] => 1.0
[0, 1] => 1.0
[1, 0] => 2.0
[1, 1] => 2.0
[2, 0] => 3.0
[2, 1] => 3.0


Another example. This time we view $\begin{bmatrix} 1 & 2 & 3 & 4\end{bmatrix}$ as $\begin{bmatrix} 1 & 2 & 3 & 4 \\ 1 & 2 & 3 & 4 \end{bmatrix}$.

In [3]:
t = Tensor.from_list([ 1, 2, 3, 4])
shape = [2,4]

for j in range(shape[1]):
    for i in range(shape[0]):
        idx = [i, j]
        print(idx, '=>', t.elem_at(shape, idx))

del t

[0, 0] => 1.0
[1, 0] => 1.0
[0, 1] => 2.0
[1, 1] => 2.0
[0, 2] => 3.0
[1, 2] => 3.0
[0, 3] => 4.0
[1, 3] => 4.0


One more example. This time the row and columns are both extended.

In [4]:
t = Tensor.from_list([1])
shape = [2,2]

for i in range(shape[0]):
    for j in range(shape[1]):
        idx = [i, j]
        print(idx, '=>', t.elem_at(shape, idx))

del t

[0, 0] => 1.0
[0, 1] => 1.0
[1, 0] => 1.0
[1, 1] => 1.0


## Implementation

Consider the following example of outer addition operation:

![from_numpy](https://numpy.org/doc/stable/_images/broadcasting_4.png)

A possible implementation of this operation is:

In [5]:
t1 = Tensor.from_list([[0], [10], [20], [30]])
assert t1.shape == list((4, 1))

t2 = Tensor.from_list([[1, 2, 3]])
assert t2.shape == list((1, 3)) 

import numpy as np
res_shape = [4, 3]
res = np.zeros(tuple(res_shape))

# to implement efficiently, consider parallelization.
for i in range(res_shape[0]):
    for j in range(res_shape[1]):
        idx = [i, j]
        res[i][j] = t1.elem_at(res_shape, idx) + t2.elem_at(res_shape, idx)

print(res)

print((t1 + t2).shape)
print((t1 + t2).to_list())

[[ 1.  2.  3.]
 [11. 12. 13.]
 [21. 22. 23.]
 [31. 32. 33.]]
[4, 3]
[[1.0, 2.0, 3.0], [11.0, 12.0, 13.0], [21.0, 22.0, 23.0], [31.0, 32.0, 33.0]]
